In [1]:
# !./generate-all.sh

In [2]:
try:
    from pytablewriter import MarkdownTableWriter
except ImportError:
    !js-pip install -skip-install-confirmation pytablewriter==1.2.0
    from pytablewriter import MarkdownTableWriter

from IPython.display import clear_output
import pandas as pd
from pathlib import Path

clear_output()

In [19]:
def load_summary_csv(path):
    df = pd.read_csv(path)
    df = df.drop(0)
    return df


def full_comparison(heading, before_path, after_path, html_output_path):
    before = load_summary_csv(before_path)
    after = load_summary_csv(after_path)

    before_after_suffixes = "_before", "_after"
    primary_key = "pass name"
    a = before[primary_key]
    before_and_after = pd.merge(
        before, after, on=primary_key, suffixes=before_after_suffixes
    )

    def compare(fields):
        included_fields = [
            field + suffix for field in fields for suffix in before_after_suffixes
        ]
        filtered = before_and_after[[primary_key] + included_fields]
        return filtered.sort_values(included_fields, ascending=False)

    # def compare2(fields):
    #     df = before_and_after[[primary_key]]
    #     for field in fields:
    #         before, after = (field + suffix for suffix in before_after_suffixes)

    #         diff = pd.DataFrame(before_and_after[after]-before_and_after[before], columns=[after])
    #         df = pd.concat([df, diff])
    #     # filtered = before_and_after[[primary_key] + included_fields]
    #     return df.sort_values([f for f in df if f != primary_key], ascending=False)

    comparison_columns = [
        "num_instructions",
        "block_duplicate_reload",
        "block_duplicate_spill",
        "reload",
        "spill",
        "time",
    ]
    comparison = compare(comparison_columns)
    html_output_path = Path(html_output_path)
    comparison.to_html(
        html_output_path.with_name(
            html_output_path.stem + "-full" + html_output_path.suffix
        ),
        index=False,
    )
    # print("Outputted HTML to ", html_output_path)
    # comparison.to_csv(html_output_path.with_suffix(".csv"), index=False)

    # cols = [
    #     # "block_duplicate_reload",
    #     # "num_instructions",
    #     # "block_duplicate_spill",
    #     # "average_temp_interf",
    #     # "spill",
    #     # "reload"
    #     # "time",
    # ]

    def compare_col(cols):
        col, other_cols = cols[0], cols[1:]
        before_col, after_col = [col + suffix for suffix in before_after_suffixes]
        other_cols = [
            col + suffix for col in other_cols for suffix in before_after_suffixes
        ]

        comparison_worse = before_and_after[
            before_and_after[after_col] > before_and_after[before_col]
        ][[primary_key, before_col, after_col] + other_cols]
        comparison_equal = before_and_after[
            before_and_after[after_col] == before_and_after[before_col]
        ][[primary_key, before_col, after_col] + other_cols]
        comparison_better = before_and_after[
            before_and_after[after_col] < before_and_after[before_col]
        ][[primary_key, before_col, after_col] + other_cols]

        col_to_output_for = "reload"
        comparison_to_output = comparison_worse
        if col == col_to_output_for:
            comparison_to_output.sort_values(
                ["num_instructions_before"], ascending=True
            ).to_html(html_output_path, index=False)

        percent = lambda comparison: round(
            len(comparison) * 100 / len(before_and_after), 2
        )
        return {
            "better (%)": percent(comparison_better),
            "worse (%)": percent(comparison_worse),
            "equal (%)": percent(comparison_equal),
        }

    cols_to_compare = [
        "reload",
        "spill",
        "block_duplicate_reload",
        "block_duplicate_spill",
        "num_instructions",
    ]
    headers = ["counter", "better (%)", "worse (%)", "equal (%)"]
    data = [
        {
            "counter": col,
            **(compare_col([col] + cols_to_compare[:i] + cols_to_compare[i + 1 :])),
        }
        for i, col in enumerate(cols_to_compare)
    ]
    matrix = [[row[key] for key in headers] for row in data]

    writer = MarkdownTableWriter(
        table_name=heading,
        headers=headers,
        value_matrix=matrix,
        margin=1,  # add a whitespace for both sides of each cell
    )
    writer.write_table()


full_comparison(
    "Files - Original vs Latest",
    "summary-original.csv",
    "summary-latest.csv",
    "/j/vdc/intern/user/mitom/pub/comparison.html",
)
full_comparison(
    "Files - Main vs Latest",
    "summary-main.csv",
    "summary-latest.csv",
    "/j/vdc/intern/user/mitom/pub/comparison.html",
)
# full_comparison(
#     "Functions - Original vs Latest",
#     "summary-original-func.csv",
#     "summary-latest-func.csv",
#     "/j/vdc/intern/user/mitom/pub/comparison-functions.html",
# )
# full_comparison(
#     "Functions - Main vs Latest",
#     "summary-main-func.csv",
#     "summary-latest-func.csv",
#     "/j/vdc/intern/user/mitom/pub/comparison-functions.html",
# )
# full_comparison(
#     "Blocks - Original vs Latest",
#     "summary-original-block.csv",
#     "summary-latest-block.csv",
#     "/j/vdc/intern/user/mitom/pub/comparison-blocks.html",
# )
# full_comparison(
#     "Blocks - Main vs Latest",
#     "summary-main-block.csv",
#     "summary-latest-block.csv",
#     "/j/vdc/intern/user/mitom/pub/comparison-blocks.html",
# )

# Files
|        counter         | better (%) | worse (%) | equal (%) |
| ---------------------- | ---------: | --------: | --------: |
| reload                 |      13.24 |      0.23 |     86.54 |
| spill                  |       2.26 |      0.68 |     97.06 |
| block_duplicate_reload |      10.29 |      0.11 |     89.59 |
| block_duplicate_spill  |       2.26 |      0.00 |     97.74 |
| num_instructions       |      13.12 |      0.45 |     86.43 |


KeyboardInterrupt: 